In [1]:
import pandas as pd
from utils import *
import lightgbm
import keras

/home/ubuntu/workspace/tools/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
mname = 'lgb'

In [2]:
df_train, df_val, df_sub = load()

df_train.shape 161570557,13
df_val.shape 23333333,13
df_sub.shape 18790469,13


In [3]:
"""
from sklearn.externals import joblib
ohe = joblib.load('../model/ohe.pickle') 
encoder = keras.models.load_model('../model/encoder')

def autoencoder(tdf):
    return encoder.predict(ohe.transform(tdf[xnames[:5]]), batch_size=256, verbose=1)

sample = 4000000

data_val = lightgbm.Dataset(autoencoder(df_val[: sample]), df_val[yname][:sample])
data = lightgbm.Dataset(autoencoder(df_train[: sample]), df_train[yname][:sample])
"""

"\nfrom sklearn.externals import joblib\nohe = joblib.load('../model/ohe.pickle') \nencoder = keras.models.load_model('../model/encoder')\n\ndef autoencoder(tdf):\n    return encoder.predict(ohe.transform(tdf[xnames[:5]]), batch_size=256, verbose=1)\n\nsample = 4000000\n\ndata_val = lightgbm.Dataset(autoencoder(df_val[: sample]), df_val[yname][:sample])\ndata = lightgbm.Dataset(autoencoder(df_train[: sample]), df_train[yname][:sample])\n"

In [4]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.1,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 5,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'scale_pos_weight': 99, # because training data is extremely unbalanced 
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
    }

In [5]:
data_val = lightgbm.Dataset(df_val[xnames], df_val[yname], categorical_feature=categorical_names)
data = lightgbm.Dataset(df_train[xnames], df_train[yname], categorical_feature=categorical_names)

In [6]:
evals_result={}
r = lightgbm.train(params,
                   data,
                   valid_sets=[data, data_val],
                   valid_names=['train', 'valid'], 
                   evals_result=evals_result,
                   num_boost_round=300,
                   early_stopping_rounds=30
                  )

/home/ubuntu/workspace/tools/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ubuntu/workspace/tools/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	train's auc: 0.953519	valid's auc: 0.955923
Training until validation scores don't improve for 30 rounds.
[2]	train's auc: 0.958712	valid's auc: 0.96026
Did not meet early stopping. Best iteration is:
[2]	train's auc: 0.958712	valid's auc: 0.96026


In [3]:
from sklearn.externals import joblib
joblib.dump(r, '../model/%s.pkl' % mname) 

In [4]:
y_val_pred = r.predict(df_val[xnames].values, num_iteration=r.best_iteration)

In [5]:
import pickle
print('writing val ...')
output_val = pd.DataFrame({'y_pred': y_val_pred, 'y': df_val[yname].astype(int).values})
with open('../data/%s_val.pickle' % mname, 'wb') as handle:
    pickle.dump(output_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

writing val ...


In [18]:
ids = df_sub['click_id'].astype(int).values
X_sub = df_sub[xnames].values
print('predicting...')
y_sub = r.predict(X_sub, num_iteration=r.best_iteration)
print('writing...')
output = pd.DataFrame({'click_id': ids, 'is_attributed': y_sub})
output.to_csv("../data/%s.csv.gz" % mname, compression='gzip', index=False)

predicting...
writing...
